In [27]:
# !pip install -U langchain
!pip install langchain langchain-community langchain-core
!pip install langchain-openai openai
!pip install langchain-deepseek
!pip install deepseek-api python-dotenv

# Installing the OpenAI integration
#!pip install -U langchain-openai
# Installing the Anthropic integration
# !pip install -U langchain-anthropic

!pip show langchain-core

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.3/397.3 kB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [29]:
import os
from langchain.tools import tool, ToolRuntime
from langchain_openai import ChatOpenAI


# 设置 DeepSeek 的 API 密钥（LangChain-OpenAI 仍然会查找 OPENAI_API_KEY）
# os.environ['DEEPSEEK_API_KEY'] = 'sk-943df854319123456789'

# 您可以尝试将 DEEPSEEK_API_KEY 的值赋给 OPENAI_API_KEY 环境变量
os.environ["OPENAI_API_KEY"] = os.getenv('DEEPSEEK_API_KEY') # 确保这个值是 DeepSeek 的 key

# 关键：指定 DeepSeek 的 API 基础 URL
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"

model = ChatOpenAI(
    model="deepseek-chat", # 使用 DeepSeek 的模型名称
    openai_api_base=DEEPSEEK_BASE_URL, # 指定 DeepSeek 的 URL
    temperature=0.7
)

In [31]:
from pydantic import BaseModel, Field  # 确保导入 Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model=model,
    # tools=tools,
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [40]:
from typing import Generic, TypeVar, Union, Callable, Literal
from pydantic import BaseModel, Field
from langchain.agents import create_agent

# Define SchemaT type variable
SchemaT = TypeVar('SchemaT', bound=BaseModel)

class ToolStrategy(Generic[SchemaT]):
    schema: type[SchemaT]
    tool_message_content: str | None = None  # Default value added for optional field
    handle_errors: Union[
        bool,
        str,
        type[Exception],
        tuple[type[Exception], ...],
        Callable[[Exception], str],
    ] = True  # Default value added for optional field

class ProductReview(BaseModel):
    """Analysis of a product review."""
    rating: int | None = Field(description="The rating of the product", ge=1, le=5)
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
    key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")

# Create agent example (ensure tools are defined)
agent = create_agent(
    model=model,
    # tools=tools,
    response_format=ProductReview  # Use ProductReview directly here
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})

print(result["structured_response"])

rating=5 sentiment='positive' key_points=['fast shipping', 'expensive']


In [41]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class MeetingAction(BaseModel):
    """Action items extracted from a meeting transcript."""
    task: str = Field(description="The specific task to be completed")
    assignee: str = Field(description="Person responsible for the task")
    priority: Literal["low", "medium", "high"] = Field(description="Priority level")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        schema=MeetingAction,
        tool_message_content="Action item captured and added to meeting notes!"
    )
)

agent.invoke({
    "messages": [{"role": "user", "content": "From our meeting: Sarah needs to update the project timeline as soon as possible"}]
})

{'messages': [HumanMessage(content='From our meeting: Sarah needs to update the project timeline as soon as possible', additional_kwargs={}, response_metadata={}, id='c89da02a-3523-4d68-b492-457aa0fd0100'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 228, 'total_tokens': 256, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 228}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '1b8f5f19-dfbd-4dfe-822e-0f11d2b05e0f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--941811c1-0095-421a-8861-084a93e0b153-0', tool_calls=[{'name': 'MeetingAction', 'args': {'task': 'update the project timeline', 'assignee': 'Sarah', 'priority': 'high'}, 'id': 'call_00_b7AOOQlFjLhPt3gOICywUS8B', 'type': 'tool_call'}], 

In [42]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    email: str = Field(description="Email address")

class EventDetails(BaseModel):
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Event date")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(Union[ContactInfo, EventDetails])  # Default: handle_errors=True
)

agent.invoke({
    "messages": [{"role": "user", "content": "Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th"}]
})

{'messages': [HumanMessage(content='Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th', additional_kwargs={}, response_metadata={}, id='c6d02905-eeb8-46f2-8903-2fcf6d12cbfc'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 253, 'total_tokens': 297, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 253}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '64c217c4-5ab1-4eed-9260-56e0d1eb3fc1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--afbd8910-95ba-46b5-8419-27626dd592b0-0', tool_calls=[{'name': 'ContactInfo', 'args': {'name': 'John Doe', 'email': 'john@email.com'}, 'id': 'call_00_SjVlJ0Rqx4O2eH65FX2HchmL', 'type': 'tool_call'}, {'name': 'EventDetails', 'args': 

In [43]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ProductRating(BaseModel):
    rating: int | None = Field(description="Rating from 1-5", ge=1, le=5)
    comment: str = Field(description="Review comment")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
    schema=ProductRating,
    handle_errors=(ValueError, TypeError)), # Retry on ValueError and TypeError
    # Default: handle_errors=True
    system_prompt="You are a helpful assistant that parses product reviews. Do not make any field or value up."
)


agent.invoke({
    "messages": [{"role": "user", "content": "Parse this: Amazing product, 10/10!"}]
})

{'messages': [HumanMessage(content='Parse this: Amazing product, 10/10!', additional_kwargs={}, response_metadata={}, id='fd95d916-4b15-4805-9f72-2b3b939d1881'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 225, 'total_tokens': 250, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 225}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '1e91ec21-c191-4076-94a5-a4decf5da532', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2991b815-b1d5-4f5b-897c-37027cf9c840-0', tool_calls=[{'name': 'ProductRating', 'args': {'rating': None, 'comment': 'Amazing product, 10/10!'}, 'id': 'call_00_Gb8PfbAgGwy7pXivZOSxdVNZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 225, 'output_tokens': 25, 'total_tokens